In [28]:
import warnings
warnings.filterwarnings("error")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
from librosa.feature import melspectrogram
from librosa import power_to_db
from librosa.display import specshow


Define some constants

In [29]:
# All EEG data is 100Hz sampling rate
FS = 100

# Length of spectrogram in seconds
SPEC_LEN = 30

# Define the data directory
data_dir = r'C:\Users\devar\Desktop\ProjectPhase1\lstm_cnn_eeg_sleep_staging\data'

# Define Mel Spectrogram parameters
n_fft = 256
hop_length = 64
n_mels = 64


Define a function to get the most frequent label within a 30-second block, if there are multiple labels. The most frequent label will apply to the entire block.

In [30]:
def get_most_frequent_label(labels):
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    most_frequent_label = unique_labels[np.argmax(label_counts)]
    return most_frequent_label


Define a function to do the spectrogram calculations

In [31]:
def calculate_spectrograms(pkl_files, data_group):
    spectrogram_list = []
    labels_list = []

    for pkl_file in pkl_files:
        df = pd.read_pickle(pkl_file)

        spectrogram_list_tmp = []
        labels_list_tmp = []

        ind = 0
        while ind < len(df):
            df_tmp = df.iloc[ind:ind+FS*SPEC_LEN]
            ch1_tmp = df_tmp['EEG Fpz-Cz'].values
            ch2_tmp = df_tmp['EEG Pz-Oz'].values
            label_tmp = get_most_frequent_label(df_tmp['label'].values)

            try:
                ch1_tmp = (ch1_tmp - np.mean(ch1_tmp)) / np.std(ch1_tmp)
                ch2_tmp = (ch2_tmp - np.mean(ch2_tmp)) / np.std(ch2_tmp)
            except Exception as e:
                spectrogram_list_tmp = []
                labels_list_tmp = []
                ind = ind + FS*SPEC_LEN
                continue

            Sxx1 = melspectrogram(y=ch1_tmp, sr=FS, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
            Sxx1 = power_to_db(Sxx1, ref=np.max)

            Sxx2 = melspectrogram(y=ch2_tmp, sr=FS, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
            Sxx2 = power_to_db(Sxx2, ref=np.max)

            spectrogram_list_tmp.append((Sxx1, Sxx2))
            labels_list_tmp.append(label_tmp)
            ind = ind + FS*SPEC_LEN

            if len(spectrogram_list_tmp) == 5:
                if -1 not in labels_list_tmp:
                    spectrogram_list.append(spectrogram_list_tmp)
                    labels_list.append(labels_list_tmp)

                spectrogram_list_tmp = []
                labels_list_tmp = []

    pickle.dump(spectrogram_list, open(f'X_{data_group}_spec.pkl', 'wb'))
    pickle.dump(labels_list, open(f'y_{data_group}.pkl', 'wb'))


Do the spectrogram calculations by train, test, and validation sets

In [34]:
pkl_files_val = [os.path.join(data_dir, 'SC4072EH.pkl'), os.path.join(data_dir, 'SC4481FV.pkl')]
pkl_files_test = [os.path.join(data_dir, 'SC4181EC.pkl')]
pkl_files_train = [
    os.path.join(data_dir, 'ST7211JJ.pkl'),
    os.path.join(data_dir, 'SC4091EC.pkl'),
    os.path.join(data_dir, 'SC4341FA.pkl'),
    os.path.join(data_dir, 'ST7051JA.pkl'),
    os.path.join(data_dir, 'ST7181JR.pkl'),
    os.path.join(data_dir, 'ST7011JP.pkl')
]

# Do spectrogram calculations
calculate_spectrograms(pkl_files_val, 'val')
calculate_spectrograms(pkl_files_test, 'test')
calculate_spectrograms(pkl_files_train, 'train')
calculate_spectrograms([os.path.join(data_dir, 'ST7152JA.pkl')], 'ST7152JA')


Exception ignored in: <_io.FileIO name='X_val_spec.pkl' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\devar\AppData\Local\Temp\ipykernel_23288\1723341290.py", line 45, in calculate_spectrograms
Exception ignored in: <_io.FileIO name='y_val.pkl' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\devar\AppData\Local\Temp\ipykernel_23288\1723341290.py", line 46, in calculate_spectrograms
Exception ignored in: <_io.FileIO name='X_test_spec.pkl' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\devar\AppData\Local\Temp\ipykernel_23288\1723341290.py", line 45, in calculate_spectrograms
Exception ignored in: <_io.FileIO name='y_test.pkl' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\devar\AppData\Local\Temp\ipykernel_23288\1723341290.py", line 46, in calculate_spectrograms
Exception ignored in: <_io.FileIO name='X_train_spec.pkl' mode='wb' closefd=True>
Traceback (most recent call la